# 🎼 인공지능 작사가 만들기

#### "단어를 넣으면 가사를(한 줄) 써주는 프로그램 제작"

### 😮 목차

**1. 데이터 준비**   
**2. 데이터 전처리**     
**3. 모델 설계, 훈련, 평가**   
**4. 가사 생성하기**   
**5. 고찰**

---

**모듈**

In [1]:
import glob
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

---

## 1. 데이터 준비

In [2]:
text_file_path = os.getenv('HOME')+'/aiffel/aiffel_exploration/lyricist/data/lyrics/*'
text_list = glob.glob(text_file_path)

raw_corpus = []
for text_file in text_list:
    with open(text_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you', 'By my side But then I spent so many nights', "Just thinking how you've done me wrong", 'I grew strong', "I learned how to get along And so you're back", 'From outer space', 'I just walked in to find you', 'Here without that look upon your face I should have changed that fucking lock', 'I would have made you leave your key', 'If I had known for just one second', "You'd be back to bother me Well now go,", 'Walk out the door', 'Just turn around', "Now, you're not welcome anymore Weren't you the one", 'Who tried to break me with desire?', "Did you think I'd crumble?", "Did you think I'd lay down and die? Oh not I,", 'I will survive', 'Yeah', "As Long as I know how to love, I know I'll be alive", "I've got all my life to live", "I've got all my love to give", 'I will survive, I will survive', 'Yeah, yeah', 'It took all the strength I had', "Just not to fall apart I'm try

---

## 2. 데이터 전처리

### 2.1 문장 정제하기

In [3]:
# 정제 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r"[' ']+", ' ', sentence) # 여러 공백은 하나로 바꾸기
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", ' ', sentence) # 해당문자 이외의 문자를 공백으로 바꾸기
    sentence = sentence.strip() # 양쪽 공백 지우기
    sentence = '<start> ' + sentence + ' <end>' # 문장의 시작과 끝에 라벨 붙이기
    return sentence

In [4]:
# 정제된 문장 모으기
corpus = []
for sentence in raw_corpus: # 한 문장씩 꺼내기
    if len(sentence) == 0 or len(sentence) == 1: # 문장의 길이가 0, 1이면 저장안함
        continue
    preprocessed_sentence = preprocess_sentence(sentence) # 문장 정제하기
    if len(preprocessed_sentence.split()) > 15: # 정제된 문장이 15단어 이상이면 저장안함
        continue     
    corpus.append(preprocessed_sentence) # 리스트 생성

print(len(corpus))
corpus[:10]

156192


['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> i would have made you leave your key <end>']

### 2.2 문장 토큰화하기

In [5]:
# 토큰화 함수
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, filters=' ', oov_token="<unk>") # 12000단어를 기억, 이미 문장 정제해서 filter 필요 없음, 12000단어 이외의 단어는 "<unk>"로 취급
    tokenizer.fit_on_texts(corpus) # 단어사전 생성
    tensor = tokenizer.texts_to_sequences(corpus) # 데이터를 벡터화
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') # 입력데이터 시퀸스 길이를 일정하게 맞춤. 시퀀스가 짧으면 문장 뒤에 패딩 붙음. padding='pre'는 앞에 붙음
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus) # 문장, 단어사전

In [6]:
# 데이터 확인
print(tensor.shape) # 15개의 단어로 구성된 156192개의 문장
print(tensor[:3])

(156192, 15)
[[   2   70  247    4   53  708    3    0    0    0    0    0    0    0
     0]
 [   2    4   53 6269    3    0    0    0    0    0    0    0    0    0
     0]
 [   2    4 1066  525    4  104   80  192  257    7    3    0    0    0
     0]]


In [7]:
# 단어사전 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 15:
        break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that


### 2.3 데이터셋 만들기

In [8]:
# 소스문장, 타겟문장 생성하기
src_input = tensor[:, :-1] # 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
tgt_input = tensor[:, 1:] # tensor에서 <start>를 잘라내서 타겟 문장 생성

print(src_input[0]) # 맨 뒤 인덱스 제거 상태
print(tgt_input[0]) # 맨 앞 인덱스 제거 상태

[  2  70 247   4  53 708   3   0   0   0   0   0   0   0]
[ 70 247   4  53 708   3   0   0   0   0   0   0   0   0]


In [9]:
# 훈련데이터, 평가데이터 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=9)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124953, 14)
Target Train: (124953, 14)


In [10]:
# 파라미터 설정
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


# 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

---

## 3. 모델 설계, 훈련, 평가

### 3.1 모델 설계

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # 단어사전의 인덱스를 해당 인덱스의 워드 벡터로 바꿈 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
vocab_size = tokenizer.num_words + 1 # 12000 + pad  
embedding_size = 256
hidden_size = 1500
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [20]:
# 본격적인 학습 전 모델에 데이터 일부 태워보기
for src_sample, tgt_sample in dataset.take(1): # take(1) 1개의 배치. 256개의 문장 데이터 가져옴
    break
    
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.28329978e-05, -2.68868316e-04, -2.59418066e-05, ...,
          6.88638247e-05, -1.90609499e-04, -2.98106715e-06],
        [ 3.49633367e-04, -4.79755865e-04,  2.33902945e-04, ...,
         -1.96660967e-05, -4.43540339e-04,  4.02360543e-04],
        [ 3.23095592e-04, -5.30259917e-04,  1.94919368e-04, ...,
         -2.16977660e-05, -4.78130125e-04,  1.00855366e-03],
        ...,
        [ 2.19061249e-03, -7.25672347e-04,  4.15295595e-04, ...,
          1.65148627e-03, -4.48797422e-04,  2.24689837e-03],
        [ 2.58490234e-03, -2.63625989e-04,  4.60571813e-04, ...,
          1.91962195e-03, -2.33269966e-04,  2.28238688e-03],
        [ 2.93728127e-03,  2.19953203e-04,  4.89130209e-04, ...,
          2.11978983e-03, -1.57870127e-05,  2.32613203e-03]],

       [[ 7.28329978e-05, -2.68868316e-04, -2.59418066e-05, ...,
          6.88638247e-05, -1.90609499e-04, -2.98106715e-06],
        [ 5.19168061e-05, -2.92468263e-04,  6

In [21]:
model.summary() # 출력 형태가 비정형. 모델이 입력 시퀀스 길이를 모르기 때문에 임의의 값을 넣어주면 설정됨

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  10542000  
_________________________________________________________________
lstm_5 (LSTM)                multiple                  18006000  
_________________________________________________________________
dense_2 (Dense)              multiple                  18013501  
Total params: 49,633,757
Trainable params: 49,633,757
Non-trainable params: 0
_________________________________________________________________


### 3.2 모델 훈련

In [22]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
488/488 [==============================] - 271s 550ms/step - loss: 4.0288
Epoch 2/10
488/488 [==============================] - 271s 554ms/step - loss: 3.0689
Epoch 3/10
488/488 [==============================] - 272s 558ms/step - loss: 2.8699
Epoch 4/10
488/488 [==============================] - 273s 559ms/step - loss: 2.7071
Epoch 5/10
488/488 [==============================] - 273s 560ms/step - loss: 2.5744
Epoch 6/10
488/488 [==============================] - 274s 560ms/step - loss: 2.4598
Epoch 7/10
488/488 [==============================] - 272s 558ms/step - loss: 2.3521
Epoch 8/10
488/488 [==============================] - 273s 558ms/step - loss: 2.2422
Epoch 9/10
488/488 [==============================] - 273s 558ms/step - loss: 2.1465
Epoch 10/10
488/488 [==============================] - 272s 558ms/step - loss: 2.0535


### 3.3 모델  평가

In [23]:
results = model.evaluate(enc_val, dec_val)
print("val_loss:", results)

977/977 [==============================] - 47s 47ms/step - loss: 2.4457
val_loss: 2.4456546306610107


---

## 4. 가사 생성하기

In [24]:
# 텍스트 생성기
def generate_text(model, tokenizer, init_sentence="<start>", max_len=17):
    # 테스트를 위해 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True: # 텍스트를 실제로 생성해야 하는 시점에서 소스문장, 타겟문장이 없다.
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 예측 단어 생성
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) # 생성 단어 이어붙이기
        if predict_word.numpy()[0] == end_token: # 마지막이 <end>면 종료
            break
        if test_tensor.shape[1] > max_len: # <start>, <end> 제외한 단어 갯수가 15를 넘기면 종료 
            break
            
    generated = ""
    for word_index in test_tensor[0].numpy(): # tokenizer를 이용해 word index를 단어로 하나씩 변환
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [25]:
# 문장 생성1
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=17)

'<start> you are the one <end> '

In [26]:
# 문장 생성2
generate_text(model, tokenizer, init_sentence="<start> no no", max_len=17)

'<start> no no no notorious <end> '

---

## 5. 고찰

- **validation loss**   
1. embedding_size를 2배로 늘렸더니 validation loss가 조금 낮아짐을 확인하였다.
2. hidden_size를 약 1.5배로 늘렸더니 validation loss가 좀 더 낮아짐을 확인하였다.
3. 목표치와 근사하게 loss값이 내려왔다. 파라미터의 크기를 좀 더 늘려 목표치 이하로 loss를 내리려다가 한시간 이상의 훈련시간을 더 기다리는게 의미가 없을 듯 싶어 멈추었다. 대신 layer에 대한 공부를 통해 다음번엔 다른 방식으로 loss값을 줄여볼 예정이다.


- **문장 생성**   
10단어 이상의 문장을 예상했으나 짧은 문장만을 생성하는 것을 확인하였다. 생각해보니 문장마다 패딩처리되는 부분이 많던 것이 기억났다. 노래 가사 자체가 짧은 문장으로 이루어져 있어 그렇다는 생각이 들었다.
<br>   

- **훈련 시간**   
처음엔 느끼지 못했는데 오늘 노드에서 모델 훈련시간이 꽤나 걸리는 것을 깨달았다. 앞으로는 훨씬 큰 데이터를 깊은 층으로 훈련시켜야 할텐데 걱정이다. colab에서 좀더 빠른 훈련이 가능한지 확인한 후 긴 훈련시간이 예상되는 노드의 경우 이곳에서 훈련시켜볼 예정이다. 훈련 시간 전에 최대한 빠르게 코드를 완성하고 휴식을 취하는 것도 좋을 듯하다.
<br>   

- **추가적으로 공부할 부분**   
validation loss를 낮추기 위해 추가적인 layer를 쌓는 방안이 있음을 알고 있으나 각 layer에 대한 이해가 아직 부족한 상황이다. 앞으로 계속 모델 훈련이 진행될것으로 생각되는만큼 layer에대한 공부를 진행할 예정이다.